# Delta Lake Lab 
## Unit 1: Create a base Parquet table
Create a base table in Parquet, off of the Kaggle Lending Club Loan dataset, preloaded into your GCS data bucket in directory parquet-source.

### 1. Imports

In [1]:
import pandas as pd
from pyspark.sql.functions import month, date_format
from pyspark.sql.types import IntegerType
from pyspark.sql import SparkSession
import warnings

warnings.filterwarnings('ignore')

### 2. Create a Spark session powered by Cloud Dataproc 

In [2]:
spark = SparkSession.builder.appName('Loan Analysis').getOrCreate()
spark

22/11/01 21:23:46 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


### 3. Declare variables

In [3]:
project_id_output = !gcloud config list --format "value(core.project)" 2>/dev/null
PROJECT_ID = project_id_output[0]
print("PROJECT_ID: ", PROJECT_ID)

PROJECT_ID:  delta-lake-lab


In [4]:
project_name_output = !gcloud projects describe $PROJECT_ID | grep name | cut -d':' -f2 | xargs
PROJECT_NAME = project_name_output[0]
print("PROJECT_NAME: ", PROJECT_NAME)

PROJECT_NAME:  delta-lake-lab


In [5]:
project_number_output = !gcloud projects describe $PROJECT_ID | grep projectNumber | cut -d':' -f2 | xargs
PROJECT_NUMBER = project_number_output[0]
print("PROJECT_NUMBER: ", PROJECT_NUMBER)

PROJECT_NUMBER:  885979867746


In [ ]:
ACCOUNT_NAME = "akhjain"

In [6]:
DATA_LAKE_ROOT_PATH= f"gs://dll-data-bucket-{PROJECT_NUMBER}-{ACCOUNT_NAME}"

In [7]:
RAW_SOURCE_FQ_GCS_PATH = f"{DATA_LAKE_ROOT_PATH}/parquet-source/*"

### 4. Explore the raw loans data

In [8]:
!gsutil ls -r $DATA_LAKE_ROOT_PATH

gs://dll-data-bucket-885979867746/parquet-source/:
gs://dll-data-bucket-885979867746/parquet-source/loans_raw_1.snappy.parquet
gs://dll-data-bucket-885979867746/parquet-source/loans_raw_2.snappy.parquet
gs://dll-data-bucket-885979867746/parquet-source/loans_raw_3.snappy.parquet


In [9]:
rawDF = spark.read.parquet(DATA_LAKE_ROOT_PATH)

In [10]:
rawDF.printSchema()

root
 |-- id: string (nullable = true)
 |-- member_id: string (nullable = true)
 |-- loan_amnt: float (nullable = true)
 |-- funded_amnt: integer (nullable = true)
 |-- funded_amnt_inv: double (nullable = true)
 |-- term: string (nullable = true)
 |-- int_rate: string (nullable = true)
 |-- installment: double (nullable = true)
 |-- grade: string (nullable = true)
 |-- sub_grade: string (nullable = true)
 |-- emp_title: string (nullable = true)
 |-- emp_length: string (nullable = true)
 |-- home_ownership: string (nullable = true)
 |-- annual_inc: float (nullable = true)
 |-- verification_status: string (nullable = true)
 |-- loan_status: string (nullable = true)
 |-- pymnt_plan: string (nullable = true)
 |-- url: string (nullable = true)
 |-- desc: string (nullable = true)
 |-- purpose: string (nullable = true)
 |-- title: string (nullable = true)
 |-- zip_code: string (nullable = true)
 |-- addr_state: string (nullable = true)
 |-- dti: float (nullable = true)
 |-- delinq_2yrs: float

In [11]:
rawDF=rawDF.na.drop(subset=["addr_state"])
rawDF.createOrReplaceTempView("loans_raw")

22/11/01 21:23:58 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


In [12]:
# Count total loans
spark.sql("select addr_state as state,loan_status, count(*) as loan_count from loans_raw group by addr_state,loan_status").show()

ANTLR Tool version 4.8 used for code generation does not match the current runtime version 4.9.3
ANTLR Runtime version 4.8 used for parser compilation does not match the current runtime version 4.9.3
ANTLR Tool version 4.8 used for code generation does not match the current runtime version 4.9.3
ANTLR Runtime version 4.8 used for parser compilation does not match the current runtime version 4.9.3


+-----+------------------+----------+
|state|       loan_status|loan_count|
+-----+------------------+----------+
|   CA|        Fully Paid|     40488|
|   CO|           Current|      7648|
|   CO|        Fully Paid|      6379|
|   NC|           Current|     10910|
|   NC| Late (16-30 days)|       102|
|   KY| Late (16-30 days)|        22|
|   PA|Late (31-120 days)|       360|
|   IN|        Fully Paid|      4011|
|   ME|   In Grace Period|        19|
|   WY|        Fully Paid|       628|
|   CO|   In Grace Period|       117|
|   ND|        Fully Paid|       178|
|   DE|Late (31-120 days)|        29|
|   TX|        Fully Paid|     21206|
|   DC|        Fully Paid|       783|
|   MN|       Charged Off|      1280|
|   IL|   In Grace Period|       276|
|   OK|   In Grace Period|        53|
|   VT|       Charged Off|        86|
|   TX|   In Grace Period|       544|
+-----+------------------+----------+
only showing top 20 rows



In [13]:
# How many distinct states?
spark.sql("select count(distinct addr_state) from loans_raw").show(truncate=False)

+--------------------------+
|count(DISTINCT addr_state)|
+--------------------------+
|52                        |
+--------------------------+



### 5. Cleanse the raw data

In [14]:
# Distinct states
spark.sql("select distinct addr_state from loans_raw").collect()

[Row(addr_state='AZ'),
 Row(addr_state='SC'),
 Row(addr_state='LA'),
 Row(addr_state='MN'),
 Row(addr_state='NJ'),
 Row(addr_state='DC'),
 Row(addr_state='OR'),
 Row(addr_state='VA'),
 Row(addr_state='RI'),
 Row(addr_state='WY'),
 Row(addr_state='KY'),
 Row(addr_state='NH'),
 Row(addr_state='MI'),
 Row(addr_state='NV'),
 Row(addr_state='WI'),
 Row(addr_state='ID'),
 Row(addr_state='CA'),
 Row(addr_state='CT'),
 Row(addr_state='NE'),
 Row(addr_state='MT'),
 Row(addr_state='NC'),
 Row(addr_state='VT'),
 Row(addr_state='MD'),
 Row(addr_state='DE'),
 Row(addr_state='MO'),
 Row(addr_state='IL'),
 Row(addr_state='ME'),
 Row(addr_state='WA'),
 Row(addr_state='ND'),
 Row(addr_state='MS'),
 Row(addr_state='AL'),
 Row(addr_state='IN'),
 Row(addr_state='OH'),
 Row(addr_state='TN'),
 Row(addr_state='NM'),
 Row(addr_state='PA'),
 Row(addr_state='SD'),
 Row(addr_state='NY'),
 Row(addr_state='TX'),
 Row(addr_state='WV'),
 Row(addr_state='GA'),
 Row(addr_state='MA'),
 Row(addr_state='KS'),
 Row(addr_s

In [15]:
# Remove data with invalid states
cleasedSubsettedDF=spark.sql("select * from loans_raw where addr_state not in ('531xx','debt_consolidation')")

In [16]:
# Quick counts
count1=cleasedSubsettedDF.count()
print(f"Cleansed and subsetted row count={count1}")

count2=cleasedSubsettedDF.select("addr_state").distinct().count()
print(f"Cleansed and subsetted distinct state count={count2}")

Cleansed and subsetted row count=740120


Cleansed and subsetted distinct state count=51


### 6. Persist the cleansed data to the data lake, as Parquet & create an external table definition on it

In [17]:
# Persist the cleaned data
cleasedSubsettedDF.coalesce(3).write.format("parquet").mode("overwrite").save(f"{DATA_LAKE_ROOT_PATH}/parquet-cleansed")

In [19]:
# Check if we are using the Dataproc Metastore
spark.sparkContext._conf.get("spark.hive.metastore.uris")

'thrift://10.87.192.20:9080'

In [20]:
# Create a database if it does not exist already
spark.sql("SHOW DATABASES;").show(truncate=False)

ivysettings.xml file not found in HIVE_HOME or HIVE_CONF_DIR,/etc/spark/conf/ivysettings.xml will be used


+---------+
|namespace|
+---------+
|default  |
+---------+



In [21]:
# Create a database if it does not exist already
spark.sql("CREATE DATABASE IF NOT EXISTS loan_db;").show(truncate=False)

++
||
++
++



In [22]:
# Create an external table defintion on the parquet files
spark.sql("DROP TABLE IF EXISTS loan_db.loans_cleansed_parquet;").show(truncate=False)
spark.sql(f"CREATE TABLE loan_db.loans_cleansed_parquet USING parquet LOCATION '{DATA_LAKE_ROOT_PATH}/parquet-cleansed';").show(truncate=False)

++
||
++
++



22/11/01 21:25:00 WARN SessionState: METASTORE_FILTER_HOOK will be ignored, since hive.security.authorization.manager is set to instance of HiveAuthorizerFactory.


++
||
++
++



In [23]:
# Review what's in the data lake
!gsutil ls -r $DATA_LAKE_ROOT_PATH

gs://dll-data-bucket-885979867746/parquet-cleansed/:
gs://dll-data-bucket-885979867746/parquet-cleansed/
gs://dll-data-bucket-885979867746/parquet-cleansed/_SUCCESS
gs://dll-data-bucket-885979867746/parquet-cleansed/part-00000-e954d40d-3eae-4602-9cd2-a326b8b18438-c000.snappy.parquet
gs://dll-data-bucket-885979867746/parquet-cleansed/part-00001-e954d40d-3eae-4602-9cd2-a326b8b18438-c000.snappy.parquet

gs://dll-data-bucket-885979867746/parquet-source/:
gs://dll-data-bucket-885979867746/parquet-source/loans_raw_1.snappy.parquet
gs://dll-data-bucket-885979867746/parquet-source/loans_raw_2.snappy.parquet
gs://dll-data-bucket-885979867746/parquet-source/loans_raw_3.snappy.parquet


### 7. Create a parquet table on the base parquet dataset

In [24]:
# Remove any residual files from potential prior run
!gsutil rm -rf $DATA_LAKE_ROOT_PATH/parquet-consumable

CommandException: 1 files/objects could not be removed.


In [25]:
# Create table in Parquet off of the cleansed raw data
spark.sql("DROP TABLE IF EXISTS loan_db.loans_by_state_parquet;").show(truncate=False)
spark.sql(f"CREATE TABLE loan_db.loans_by_state_parquet USING parquet LOCATION '{DATA_LAKE_ROOT_PATH}/parquet-consumable' AS SELECT addr_state, count(loan_status) as count FROM loan_db.loans_cleansed_parquet GROUP BY addr_state;")

++
||
++
++



DataFrame[]

In [26]:
# Check the Dataproc metastore for the new table
spark.sql("show tables from loan_db;").show(truncate=False)

+---------+----------------------+-----------+
|namespace|tableName             |isTemporary|
+---------+----------------------+-----------+
|loan_db  |loans_by_state_parquet|false      |
|loan_db  |loans_cleansed_parquet|false      |
|         |loans_raw             |false      |
+---------+----------------------+-----------+



In [27]:
# List some data
spark.sql("select * from loan_db.loans_by_state_parquet").show(truncate=False)

+----------+------+
|addr_state|count |
+----------+------+
|AZ        |17257 |
|SC        |9020  |
|LA        |8728  |
|MN        |13501 |
|NJ        |27212 |
|DC        |1789  |
|OR        |8696  |
|VA        |21271 |
|RI        |3307  |
|WY        |1635  |
|KY        |7147  |
|NH        |3594  |
|MI        |19324 |
|NV        |10446 |
|WI        |9741  |
|ID        |876   |
|CA        |103526|
|CT        |11338 |
|NE        |2104  |
|MT        |2082  |
+----------+------+
only showing top 20 rows



### 8. Review what is in the data lake

Review cell #8. There was just one directory - parquet-source. 

Next review cell #19. A directory called parquet-cleased was added. 

At the end of this notebook, we also have a parquet-cleansed directory

In [28]:
!gsutil ls -r $DATA_LAKE_ROOT_PATH

gs://dll-data-bucket-885979867746/parquet-cleansed/:
gs://dll-data-bucket-885979867746/parquet-cleansed/
gs://dll-data-bucket-885979867746/parquet-cleansed/_SUCCESS
gs://dll-data-bucket-885979867746/parquet-cleansed/part-00000-e954d40d-3eae-4602-9cd2-a326b8b18438-c000.snappy.parquet
gs://dll-data-bucket-885979867746/parquet-cleansed/part-00001-e954d40d-3eae-4602-9cd2-a326b8b18438-c000.snappy.parquet

gs://dll-data-bucket-885979867746/parquet-consumable/:
gs://dll-data-bucket-885979867746/parquet-consumable/
gs://dll-data-bucket-885979867746/parquet-consumable/_SUCCESS
gs://dll-data-bucket-885979867746/parquet-consumable/part-00000-707d61b6-b488-4daa-b9c1-f1dc435eded2-c000.snappy.parquet

gs://dll-data-bucket-885979867746/parquet-source/:
gs://dll-data-bucket-885979867746/parquet-source/loans_raw_1.snappy.parquet
gs://dll-data-bucket-885979867746/parquet-source/loans_raw_2.snappy.parquet
gs://dll-data-bucket-885979867746/parquet-source/loans_raw_3.snappy.parquet


We will use the data under the parquet-consumable directory in the next unit, and create a Delta table off of it.

### THIS CONCLUDES THIS UNIT. PROCEED TO THE NEXT NOTEBOOK